In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [6]:
df = pd.read_csv('smile-annotations-final.csv', names = ['id', 'text', 'category'])
df.set_index('id', inplace = True)

In [7]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [8]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [9]:
# Removing all the tweets with multiple categories and also removing the nocode category tweets

In [10]:
df = df[~df.category.str.contains('\|')]

In [11]:
df = df[df.category != 'nocode']

In [12]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [16]:
labels = df.category.unique()

In [18]:
label_dict = {}
for index, labels in enumerate(labels):
    label_dict[labels] = index
    

In [19]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [20]:
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


# Training and Validation Split

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
# Addressing the problem of non-uniform class size

In [23]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.15,
    random_state = 17,
    stratify = df.label.values
)

In [24]:
df['data_type'] = ['not_set']*df.shape[0]

In [28]:
#Adding the type of data(train or test) label to the dataframe

In [26]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [27]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train


In [30]:
# grouping category, labels, data types in labels for their counts

In [29]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

# Loading Tokenizer and Encoding our data

In [32]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [33]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True,
    
)

In [46]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens = True,
    return_attention_mask = True, 
    pad_to_max_length = True,
    max_length = 256,
    truncation = True,
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens = True,
    return_attention_mask = True, 
    pad_to_max_length = True,
    max_length = 256,
    truncation=True,
    return_tensors = 'pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)

In [47]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [48]:
print(len(dataset_train))
print(len(dataset_val))

1258
223


In [49]:
# Setting up BERT Pretrained model

In [50]:
from transformers import BertForSequenceClassification

In [53]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Creating Data Loaders

In [54]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [58]:
batch_size = 16

dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size = batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler = RandomSampler(dataset_val),
    batch_size = 32
)

# Setting up Optimizer and Scheduler

In [60]:
# Import Adam with Weight Decay
from transformers import AdamW, get_linear_schedule_with_warmup

In [62]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, #2e-5 > 5e-5
    eps = 1e-8
)

In [63]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train) * epochs,
)

# Defining our Performance Metrics 

In [64]:
import numpy as np

In [65]:
from sklearn.metrics import f1_score

In [66]:
# This is what we'll get
#preds = [ 0.9 0.5 0.4 0 0 0] 
#This is what we want 
#preds = [1 0 0 0 0 0]

In [68]:
# We are going to use Accuracy metric approach 
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [72]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k,v in labels_dict.items()}
    
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_pred[y_preds == label])/len(y_true)}')

# Creating our Training Loop

In [75]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [76]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
print(device)

cpu


In [78]:
def evaluate(dataloader_val):
    
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids': batch[0],
            'attention_masks': batch[1],
            'labels': batch[2],
        }
        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        total_val_loss += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['lables'].cpu().numpy()
        predicts.append(logits)
        true_vals.append(label_ids)
        
    loss_val_average = total_val_loss / len(dataloader_val)
    
    predictions = np.concatenate(predictions, axis = 0)
    true_vals = np.concatenate(true_vals, axis = 0)
    
    return loss_val_average, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, desc='Epoch: {:1d}'.format(epoch),
                       leave = False,
                       disable = False)
    
    for batch in progress_bar:
        
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm(model.parameters, 1.0)
        
        optmizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training loss':'{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'Models/ft_epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_average = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_average}')
    
    val_loss, predictions, true_val = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 score (weighted): {val_f1}')
    
    
    
        